# K-Fold Cross-Validation Classification Example

This notebook demonstrates how to use the `kfold_classification.py` script for patch-level invasion classification with sample-level splitting.

## 1. Basic Usage

Run the classification with default settings (5 folds, logistic regression, no tissue filtering):

In [ ]:
# Basic example with 5-fold cross-validation
!python ../scripts/kfold_classification.py \
    --h5_path "K:/499-ProjectData/2025/P25-0048_Thyroid_Recurrence/06-UNI-adaptation/embeddings/test_thyroid_embeddings.h5" \
    --n_folds 5

## 2. With Tissue Filtering

Only include patches with at least 50% tissue content:

In [ ]:
# With tissue threshold
!python ../scripts/kfold_classification.py \
    --h5_path "K:/499-ProjectData/2025/P25-0048_Thyroid_Recurrence/06-UNI-adaptation/embeddings/test_thyroid_embeddings.h5" \
    --n_folds 5 \
    --tissue_threshold 50

## 3. Different Classifiers

Try different classification algorithms:

In [ ]:
# Random Forest
!python ../scripts/kfold_classification.py \
    --h5_path "K:/499-ProjectData/2025/P25-0048_Thyroid_Recurrence/06-UNI-adaptation/embeddings/test_thyroid_embeddings.h5" \
    --n_folds 5 \
    --classifier random_forest \
    --output_dir "results/classification/random_forest"

In [ ]:
# Gradient Boosting
!python ../scripts/kfold_classification.py \
    --h5_path "K:/499-ProjectData/2025/P25-0048_Thyroid_Recurrence/06-UNI-adaptation/embeddings/test_thyroid_embeddings.h5" \
    --n_folds 5 \
    --classifier gradient_boosting \
    --output_dir "results/classification/gradient_boosting"

## 4. Loading and Analyzing Results

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load fold-level results
fold_results = pd.read_csv("results/classification/cv_results_logistic_k5_tissue0_folds.csv")
print("Fold-level Results:")
print(fold_results)

# Load aggregated results
agg_results = pd.read_csv("results/classification/cv_results_logistic_k5_tissue0_aggregated.csv")
print("\nAggregated Results:")
print(agg_results)

# Load complete results (includes confusion matrices)
with open("results/classification/cv_results_logistic_k5_tissue0_complete.json", "r") as f:
    complete_results = json.load(f)

print("\nConfiguration:")
for key, value in complete_results["configuration"].items():
    print(f"  {key}: {value}")

## 5. Custom Analysis

Create custom visualizations from the results:

In [ ]:
# Compare metrics across folds
fig, ax = plt.subplots(figsize=(10, 6))

metrics = ['accuracy', 'precision', 'recall', 'f1']
fold_results[metrics].plot(kind='bar', ax=ax)

ax.set_xlabel('Fold')
ax.set_ylabel('Score')
ax.set_title('Metrics Comparison Across Folds')
ax.set_xticklabels([f"Fold {i+1}" for i in range(len(fold_results))], rotation=0)
ax.legend(loc='lower right')
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

## 6. Programmatic Usage

You can also use the `PatchClassifier` class directly in Python:

In [ ]:
import sys
sys.path.append('../scripts')

from kfold_classification import PatchClassifier
from pathlib import Path

# Create classifier instance
classifier = PatchClassifier(
    h5_path="K:/499-ProjectData/2025/P25-0048_Thyroid_Recurrence/06-UNI-adaptation/embeddings/test_thyroid_embeddings.h5",
    n_folds=5,
    tissue_threshold=30.0,  # Only patches with >= 30% tissue
    classifier_type="logistic",
    random_state=42,
    scale_features=True
)

# Run the full pipeline
classifier.run(output_dir=Path("results/classification/custom"))

# Access results directly
print("\nAggregated Metrics:")
for key, value in classifier.aggregated_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## 7. Comparing Multiple Experiments

Compare results from different tissue thresholds or classifiers:

In [ ]:
# Run experiments with different tissue thresholds
thresholds = [0, 20, 40, 60, 80]

comparison_results = []

for threshold in thresholds:
    print(f"\nRunning experiment with tissue_threshold={threshold}...")
    
    classifier = PatchClassifier(
        h5_path="K:/499-ProjectData/2025/P25-0048_Thyroid_Recurrence/06-UNI-adaptation/embeddings/test_thyroid_embeddings.h5",
        n_folds=5,
        tissue_threshold=float(threshold),
        classifier_type="logistic",
        random_state=42
    )
    
    try:
        classifier.run(output_dir=Path(f"results/classification/threshold_{threshold}"))
        
        # Store results
        comparison_results.append({
            'tissue_threshold': threshold,
            'accuracy': classifier.aggregated_metrics['accuracy_mean'],
            'precision': classifier.aggregated_metrics['precision_mean'],
            'recall': classifier.aggregated_metrics['recall_mean'],
            'f1': classifier.aggregated_metrics['f1_mean'],
            'auc_roc': classifier.aggregated_metrics['auc_roc_mean'],
        })
    except Exception as e:
        print(f"  Failed: {e}")

# Create comparison DataFrame
df_comparison = pd.DataFrame(comparison_results)
print("\nComparison Results:")
print(df_comparison)

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 6))

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc_roc']
for metric in metrics_to_plot:
    if metric in df_comparison.columns:
        ax.plot(df_comparison['tissue_threshold'], df_comparison[metric], 
                marker='o', label=metric.capitalize(), linewidth=2)

ax.set_xlabel('Tissue Threshold (%)', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Classification Performance vs Tissue Threshold', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)

plt.tight_layout()
plt.savefig('results/classification/threshold_comparison.png', dpi=150, bbox_inches='tight')
plt.show()